# Day12_2: 모델 평가 (Model Evaluation)

## 📚 학습 목표

**Part 1: 분류 평가 지표**
1. Accuracy (정확도) 이해하기
2. Precision (정밀도)과 Recall (재현율) 구분하기
3. F1-score 계산 원리 파악하기
4. Confusion Matrix (혼동행렬) 해석하기
5. Plotly로 혼동행렬 시각화하기

**Part 2: 회귀 평가 지표 및 과적합 방지**
1. MAE, MSE, RMSE 계산하기
2. R-squared (결정계수) 해석하기
3. 교차 검증(Cross Validation) 적용하기
4. Learning Curve로 과적합/과소적합 진단하기

---

## 🎯 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| 정확도 | 전체 성능 요약 | "모델 정확도 95%" 보고 |
| 정밀도/재현율 | 비즈니스 목표 맞춤 | 스팸 필터 vs 암 진단 |
| F1-score | 불균형 데이터 평가 | 이탈 예측 (이탈자 5%) |
| 혼동행렬 | 오분류 유형 분석 | FP/FN 비용 계산 |
| RMSE/MAE | 예측 오차 크기 | 가격 예측 오차 |
| R-squared | 설명력 평가 | "모델이 변동의 80% 설명" |
| 교차검증 | 과적합 방지 | 안정적인 성능 추정 |
| Learning Curve | 진단 시각화 | 학습 데이터 양 결정 |

**분석가 관점**: 모델을 "얼마나 잘 만들었나?"를 숫자로 증명하는 것이 평가 지표입니다. 어떤 지표를 선택하느냐에 따라 비즈니스 임팩트가 달라집니다!

---

In [ ]:
# 필수 라이브러리 임포트
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# scikit-learn 평가 도구
from sklearn.model_selection import train_test_split, cross_val_score, learning_curve
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report,
    mean_absolute_error, mean_squared_error, r2_score
)

# 분류/회귀 모델
from sklearn.datasets import load_breast_cancer, load_diabetes
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import StandardScaler

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

print("라이브러리 로드 완료!")

---

# Part 1: 분류 평가 지표

---

## 1.1 데이터 준비

유방암 데이터셋을 사용하여 분류 모델을 학습하고 평가합니다.

In [ ]:
# 유방암 데이터 로드
cancer = load_breast_cancer()
X = pd.DataFrame(cancer.data, columns=cancer.feature_names)
y = cancer.target  # 0: 악성, 1: 양성

print(f"데이터 크기: {X.shape}")
print(f"타겟 분포: 악성={sum(y==0)}, 양성={sum(y==1)}")

In [ ]:
# 학습/테스트 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"학습 데이터: {X_train.shape[0]}개")
print(f"테스트 데이터: {X_test.shape[0]}개")

In [ ]:
# 로지스틱 회귀 모델 학습
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_train_scaled, y_train)

# 예측
y_pred = model.predict(X_test_scaled)
print(f"예측 완료! 예측 샘플: {y_pred[:10]}")

---

## 1.2 Accuracy (정확도)

**정확도 = 맞은 예측 / 전체 예측**

가장 직관적인 지표이지만, 클래스 불균형 시 오해를 줄 수 있습니다.

In [ ]:
# 정확도 계산
accuracy = accuracy_score(y_test, y_pred)
print(f"정확도(Accuracy): {accuracy:.4f} ({accuracy*100:.2f}%)")

In [ ]:
# 수동 계산으로 이해하기
correct = sum(y_test == y_pred)
total = len(y_test)
manual_accuracy = correct / total

print(f"맞은 예측: {correct}")
print(f"전체 예측: {total}")
print(f"정확도: {correct}/{total} = {manual_accuracy:.4f}")

### 💡 실무 예시: 정확도의 함정 (클래스 불균형)

만약 100명 중 95명이 정상, 5명이 사기꾼이라면?  
"모두 정상"이라고 예측해도 정확도 95%!

In [ ]:
# 불균형 데이터 시뮬레이션
y_true_imbalanced = [0] * 95 + [1] * 5  # 95% 정상, 5% 사기
y_pred_always_zero = [0] * 100          # 무조건 "정상" 예측

accuracy_imbalanced = accuracy_score(y_true_imbalanced, y_pred_always_zero)
print(f"불균형 데이터에서 '모두 정상' 예측 정확도: {accuracy_imbalanced:.2f}")
print("=> 사기꾼을 하나도 못 잡았는데 95% 정확도! 함정!")

---

## 1.3 Precision과 Recall

| 지표 | 의미 | 공식 | 중요한 경우 |
|------|------|------|------------|
| **Precision** (정밀도) | 양성 예측 중 실제 양성 비율 | TP / (TP + FP) | 오탐(FP) 비용이 클 때 |
| **Recall** (재현율) | 실제 양성 중 양성 예측 비율 | TP / (TP + FN) | 미탐(FN) 비용이 클 때 |

**용어**:
- TP (True Positive): 양성을 양성으로 맞춤
- FP (False Positive): 음성을 양성으로 잘못 예측
- FN (False Negative): 양성을 음성으로 잘못 예측
- TN (True Negative): 음성을 음성으로 맞춤

In [ ]:
# Precision과 Recall 계산
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print(f"Precision (정밀도): {precision:.4f}")
print(f"Recall (재현율): {recall:.4f}")

### 💡 실무 예시: Precision vs Recall 선택 기준

In [ ]:
# 상황별 지표 선택
scenarios = [
    {"문제": "스팸 메일 필터", "우선 지표": "Precision", 
     "이유": "정상 메일을 스팸으로 오탐하면 중요한 메일 놓침"},
    {"문제": "암 진단 스크리닝", "우선 지표": "Recall", 
     "이유": "암환자를 정상으로 오진하면 치료 기회 상실"},
    {"문제": "신용카드 사기 탐지", "우선 지표": "Recall", 
     "이유": "사기를 놓치면 고객 피해 발생"},
    {"문제": "추천 시스템", "우선 지표": "Precision", 
     "이유": "관심 없는 상품 추천은 사용자 경험 저하"},
]

df_scenarios = pd.DataFrame(scenarios)
print(df_scenarios.to_string(index=False))

---

## 1.4 F1-Score

**F1-score = 2 * (Precision * Recall) / (Precision + Recall)**

Precision과 Recall의 조화 평균. 불균형 데이터에서 정확도 대신 사용합니다.

In [ ]:
# F1-score 계산
f1 = f1_score(y_test, y_pred)
print(f"F1-score: {f1:.4f}")

# 수동 계산으로 확인
f1_manual = 2 * (precision * recall) / (precision + recall)
print(f"수동 계산: {f1_manual:.4f}")

In [ ]:
# 전체 분류 리포트
print("=" * 60)
print("분류 리포트 (Classification Report)")
print("=" * 60)
print(classification_report(y_test, y_pred, target_names=['악성(0)', '양성(1)']))

---

## 1.5 Confusion Matrix (혼동행렬)

모델의 예측 결과를 한눈에 보여주는 표입니다.

In [ ]:
# 혼동행렬 계산
cm = confusion_matrix(y_test, y_pred)
print("혼동행렬:")
print(cm)
print()
print("해석:")
print(f"  TN (악성을 악성으로): {cm[0,0]}")
print(f"  FP (악성을 양성으로): {cm[0,1]}")
print(f"  FN (양성을 악성으로): {cm[1,0]}")
print(f"  TP (양성을 양성으로): {cm[1,1]}")

In [ ]:
# Plotly로 혼동행렬 시각화
def plot_confusion_matrix(cm, labels=['악성(0)', '양성(1)']):
    """혼동행렬을 Plotly 히트맵으로 시각화"""
    # 텍스트 어노테이션 생성
    annotations = []
    for i in range(len(cm)):
        for j in range(len(cm[0])):
            annotations.append(
                dict(
                    x=labels[j], y=labels[i],
                    text=str(cm[i][j]),
                    showarrow=False,
                    font=dict(size=20, color='white' if cm[i][j] > cm.max()/2 else 'black')
                )
            )
    
    fig = go.Figure(data=go.Heatmap(
        z=cm,
        x=labels,
        y=labels,
        colorscale='Blues',
        showscale=True
    ))
    
    fig.update_layout(
        title='Confusion Matrix (혼동행렬)',
        xaxis_title='예측값 (Predicted)',
        yaxis_title='실제값 (Actual)',
        annotations=annotations,
        width=500, height=450
    )
    
    return fig

fig = plot_confusion_matrix(cm)
fig.show()

### 💡 실무 예시: 오분류 비용 분석

In [ ]:
# 비용 분석 (암 진단 시나리오)
fp_cost = 100_000  # FP 비용: 불필요한 정밀검사 비용
fn_cost = 5_000_000  # FN 비용: 암 환자 놓침 (치료 지연, 생명 위험)

tn, fp, fn, tp = cm.ravel()

total_fp_cost = fp * fp_cost
total_fn_cost = fn * fn_cost
total_cost = total_fp_cost + total_fn_cost

print(f"FP 발생 건수: {fp}건 x {fp_cost:,}원 = {total_fp_cost:,}원")
print(f"FN 발생 건수: {fn}건 x {fn_cost:,}원 = {total_fn_cost:,}원")
print(f"총 오분류 비용: {total_cost:,}원")
print("\n=> FN 비용이 훨씬 크므로 Recall을 높이는 것이 중요!")

---

# Part 2: 회귀 평가 지표 및 과적합 방지

---

## 2.1 회귀 데이터 준비

In [ ]:
# 당뇨병 데이터 로드
diabetes = load_diabetes()
X_reg = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
y_reg = diabetes.target

print(f"데이터 크기: {X_reg.shape}")
print(f"타겟 통계: 평균={y_reg.mean():.1f}, 표준편차={y_reg.std():.1f}")

In [ ]:
# 학습/테스트 분할
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.3, random_state=42
)

# Ridge 회귀 모델 학습
reg_model = Ridge(alpha=1.0, random_state=42)
reg_model.fit(X_train_reg, y_train_reg)

# 예측
y_pred_reg = reg_model.predict(X_test_reg)
print(f"예측 완료! 샘플: {y_pred_reg[:5].round(1)}")

---

## 2.2 MAE, MSE, RMSE

| 지표 | 공식 | 특징 |
|------|------|------|
| **MAE** | 평균(\|실제-예측\|) | 이상치에 덜 민감 |
| **MSE** | 평균((실제-예측)^2) | 큰 오차에 더 큰 패널티 |
| **RMSE** | sqrt(MSE) | MSE를 원래 단위로 |


In [ ]:
# 회귀 평가 지표 계산
mae = mean_absolute_error(y_test_reg, y_pred_reg)
mse = mean_squared_error(y_test_reg, y_pred_reg)
rmse = np.sqrt(mse)

print(f"MAE  (평균 절대 오차): {mae:.2f}")
print(f"MSE  (평균 제곱 오차): {mse:.2f}")
print(f"RMSE (제곱근 MSE):    {rmse:.2f}")

In [ ]:
# 수동 계산으로 이해하기
errors = y_test_reg - y_pred_reg

mae_manual = np.mean(np.abs(errors))
mse_manual = np.mean(errors ** 2)
rmse_manual = np.sqrt(mse_manual)

print(f"MAE 수동계산: {mae_manual:.2f}")
print(f"MSE 수동계산: {mse_manual:.2f}")
print(f"RMSE 수동계산: {rmse_manual:.2f}")

### 💡 실무 예시: 지표 선택 가이드

In [ ]:
# MAE vs RMSE 비교
print("지표 선택 가이드:")
print("-" * 50)
print("MAE 사용 시:")
print("  - 이상치가 많은 데이터")
print("  - 오차 크기에 균등한 가중치 부여")
print("  - 예: 배송 시간 예측 (가끔 지연 발생)")
print()
print("RMSE 사용 시:")
print("  - 큰 오차를 더 강하게 패널티")
print("  - 예: 주가 예측 (큰 오차 = 큰 손실)")
print("  - 예: 재고 예측 (과잉재고 비용)")

---

## 2.3 R-squared (결정계수)

**R^2 = 1 - (예측 오차 분산 / 타겟 분산)**

- R^2 = 1: 완벽한 예측
- R^2 = 0: 평균만 예측하는 것과 동일
- R^2 < 0: 평균보다 못한 예측 (매우 나쁨)

In [ ]:
# R-squared 계산
r2 = r2_score(y_test_reg, y_pred_reg)
print(f"R-squared: {r2:.4f}")
print(f"=> 모델이 타겟 변동의 {r2*100:.1f}%를 설명합니다.")

In [ ]:
# 실제 vs 예측 시각화
fig = px.scatter(
    x=y_test_reg, y=y_pred_reg,
    labels={'x': '실제값', 'y': '예측값'},
    title=f'실제 vs 예측 (R² = {r2:.3f})'
)

# 완벽한 예측선 (y=x)
min_val, max_val = min(y_test_reg), max(y_test_reg)
fig.add_trace(go.Scatter(
    x=[min_val, max_val], y=[min_val, max_val],
    mode='lines', name='완벽한 예측',
    line=dict(color='red', dash='dash')
))

fig.update_layout(width=600, height=500)
fig.show()

---

## 2.4 교차 검증 (Cross Validation)

단일 학습/테스트 분할의 문제:
- 어떤 분할이냐에 따라 성능이 달라짐
- 과적합 여부 판단 어려움

**K-Fold 교차 검증**:
- 데이터를 K개로 나눠 K번 학습/평가
- 평균 성능과 표준편차 확인

In [ ]:
# 분류 모델 교차검증 (정확도)
cv_scores_clf = cross_val_score(
    LogisticRegression(random_state=42, max_iter=1000),
    scaler.fit_transform(X), y,
    cv=5,  # 5-fold
    scoring='accuracy'
)

print("분류 모델 5-Fold 교차검증 (정확도):")
print(f"각 Fold: {cv_scores_clf.round(4)}")
print(f"평균: {cv_scores_clf.mean():.4f} (+/- {cv_scores_clf.std()*2:.4f})")

In [ ]:
# 회귀 모델 교차검증 (R-squared)
cv_scores_reg = cross_val_score(
    Ridge(alpha=1.0, random_state=42),
    X_reg, y_reg,
    cv=5,
    scoring='r2'
)

print("회귀 모델 5-Fold 교차검증 (R²):")
print(f"각 Fold: {cv_scores_reg.round(4)}")
print(f"평균: {cv_scores_reg.mean():.4f} (+/- {cv_scores_reg.std()*2:.4f})")

### 💡 실무 예시: 다양한 scoring 옵션

In [ ]:
# 분류에서 F1-score로 교차검증
cv_f1 = cross_val_score(
    LogisticRegression(random_state=42, max_iter=1000),
    scaler.fit_transform(X), y,
    cv=5,
    scoring='f1'  # F1-score
)
print(f"F1-score 교차검증: {cv_f1.mean():.4f} (+/- {cv_f1.std()*2:.4f})")

# 회귀에서 RMSE로 교차검증 (negative이므로 부호 변환)
cv_rmse = cross_val_score(
    Ridge(alpha=1.0),
    X_reg, y_reg,
    cv=5,
    scoring='neg_root_mean_squared_error'
)
print(f"RMSE 교차검증: {-cv_rmse.mean():.2f} (+/- {cv_rmse.std()*2:.2f})")

---

## 2.5 Learning Curve (학습 곡선)

학습 데이터 크기에 따른 성능 변화를 보여줍니다.

**진단 패턴**:
- **과소적합**: 학습/검증 점수 모두 낮음
- **과적합**: 학습 점수 높고, 검증 점수 낮음 (큰 갭)
- **적합**: 학습/검증 점수 모두 높고, 갭이 작음

In [ ]:
# 학습 곡선 계산
train_sizes, train_scores, val_scores = learning_curve(
    LogisticRegression(random_state=42, max_iter=1000),
    scaler.fit_transform(X), y,
    train_sizes=np.linspace(0.1, 1.0, 10),
    cv=5,
    scoring='accuracy'
)

# 평균과 표준편차
train_mean = train_scores.mean(axis=1)
train_std = train_scores.std(axis=1)
val_mean = val_scores.mean(axis=1)
val_std = val_scores.std(axis=1)

print(f"학습 데이터 크기: {train_sizes}")

In [ ]:
# Plotly로 Learning Curve 시각화
fig = go.Figure()

# 학습 점수
fig.add_trace(go.Scatter(
    x=train_sizes, y=train_mean,
    mode='lines+markers',
    name='학습 점수',
    line=dict(color='blue'),
    error_y=dict(type='data', array=train_std, visible=True)
))

# 검증 점수
fig.add_trace(go.Scatter(
    x=train_sizes, y=val_mean,
    mode='lines+markers',
    name='검증 점수',
    line=dict(color='green'),
    error_y=dict(type='data', array=val_std, visible=True)
))

fig.update_layout(
    title='Learning Curve (학습 곡선)',
    xaxis_title='학습 데이터 크기',
    yaxis_title='정확도',
    width=700, height=450,
    yaxis=dict(range=[0.9, 1.0])
)

fig.show()

### 💡 실무 예시: 과적합 모델 vs 정상 모델 비교

In [ ]:
# 과적합 모델 (복잡한 Random Forest)
train_sizes_of, train_scores_of, val_scores_of = learning_curve(
    RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42),
    scaler.fit_transform(X), y,
    train_sizes=np.linspace(0.1, 1.0, 10),
    cv=5
)

# 정규화된 모델 (제한된 깊이)
train_sizes_ok, train_scores_ok, val_scores_ok = learning_curve(
    RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42),
    scaler.fit_transform(X), y,
    train_sizes=np.linspace(0.1, 1.0, 10),
    cv=5
)

In [ ]:
# 비교 시각화
fig = make_subplots(rows=1, cols=2, subplot_titles=['과적합 위험 (깊이 무제한)', '정규화됨 (깊이=5)'])

# 과적합 모델
fig.add_trace(go.Scatter(x=train_sizes_of, y=train_scores_of.mean(axis=1),
                         name='학습(과적합)', line=dict(color='red')), row=1, col=1)
fig.add_trace(go.Scatter(x=train_sizes_of, y=val_scores_of.mean(axis=1),
                         name='검증(과적합)', line=dict(color='red', dash='dash')), row=1, col=1)

# 정상 모델
fig.add_trace(go.Scatter(x=train_sizes_ok, y=train_scores_ok.mean(axis=1),
                         name='학습(정상)', line=dict(color='blue')), row=1, col=2)
fig.add_trace(go.Scatter(x=train_sizes_ok, y=val_scores_ok.mean(axis=1),
                         name='검증(정상)', line=dict(color='blue', dash='dash')), row=1, col=2)

fig.update_layout(height=400, width=900, title_text='과적합 vs 정상 모델 Learning Curve')
fig.update_yaxes(range=[0.9, 1.0])
fig.show()

---

## 2.6 모델 비교 대시보드

In [ ]:
# 여러 분류 모델 비교
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'SVM': SVC(random_state=42)
}

results = []
for name, model in models.items():
    cv_acc = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='accuracy')
    cv_f1 = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='f1')
    
    results.append({
        'Model': name,
        'Accuracy': f"{cv_acc.mean():.4f} (+/- {cv_acc.std():.4f})",
        'F1-Score': f"{cv_f1.mean():.4f} (+/- {cv_f1.std():.4f})",
        'Acc_Mean': cv_acc.mean(),
        'F1_Mean': cv_f1.mean()
    })

results_df = pd.DataFrame(results)
print(results_df[['Model', 'Accuracy', 'F1-Score']].to_string(index=False))

In [ ]:
# 시각적 비교
fig = go.Figure()

fig.add_trace(go.Bar(
    name='Accuracy',
    x=results_df['Model'],
    y=results_df['Acc_Mean'],
    marker_color='steelblue'
))

fig.add_trace(go.Bar(
    name='F1-Score',
    x=results_df['Model'],
    y=results_df['F1_Mean'],
    marker_color='darkorange'
))

fig.update_layout(
    title='모델 성능 비교',
    yaxis_title='점수',
    barmode='group',
    yaxis=dict(range=[0.9, 1.0]),
    width=700, height=400
)

fig.show()

---

## 🎯 실습 퀴즈

**난이도**: ⭐ (쉬움) ~ ⭐⭐⭐⭐⭐ (어려움)

---

### Q1. 정확도 계산하기 ⭐

**문제**: 다음 실제값과 예측값으로 정확도(Accuracy)를 계산하세요.

```python
y_true = [1, 0, 1, 1, 0, 1, 0, 0, 1, 1]
y_pred = [1, 0, 1, 0, 0, 1, 1, 0, 1, 0]
```

In [ ]:
y_true = [1, 0, 1, 1, 0, 1, 0, 0, 1, 1]
y_pred = [1, 0, 1, 0, 0, 1, 1, 0, 1, 0]

# 여기에 코드를 작성하세요


### Q2. 혼동행렬 해석하기 ⭐⭐

**문제**: Q1의 데이터로 혼동행렬을 만들고, TP, FP, FN, TN 값을 출력하세요.

**힌트**: confusion_matrix()와 .ravel() 사용

In [ ]:
# 여기에 코드를 작성하세요


### Q3. Precision/Recall 계산하기 ⭐⭐

**문제**: Q1의 데이터로 Precision과 Recall을 계산하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q4. 회귀 지표 계산하기 ⭐⭐

**문제**: 다음 실제값과 예측값으로 MAE, RMSE, R-squared를 계산하세요.

```python
y_true_reg = [100, 150, 200, 180, 220]
y_pred_reg = [110, 140, 195, 175, 230]
```

In [ ]:
y_true_reg = [100, 150, 200, 180, 220]
y_pred_reg = [110, 140, 195, 175, 230]

# 여기에 코드를 작성하세요


### Q5. F1-score 수동 계산하기 ⭐⭐

**문제**: Precision=0.8, Recall=0.6일 때 F1-score를 수동으로 계산하세요.

**공식**: F1 = 2 * (P * R) / (P + R)

In [ ]:
precision = 0.8
recall = 0.6

# 여기에 코드를 작성하세요


### Q6. 교차검증 수행하기 ⭐⭐⭐

**문제**: 유방암 데이터에 대해 RandomForestClassifier로 5-fold 교차검증을 수행하고, 평균 정확도와 표준편차를 출력하세요.

In [ ]:
# 데이터는 이미 로드됨: X, y (유방암 데이터)

# 여기에 코드를 작성하세요


### Q7. 지표 선택 논리 ⭐⭐⭐

**문제**: 아래 상황에서 어떤 지표를 우선해야 하는지 답하고, 그 이유를 설명하세요.

1. 코로나 진단 키트: Precision vs Recall?
2. 뉴스 추천 시스템: Precision vs Recall?
3. 주가 예측 모델: MAE vs RMSE?

In [ ]:
# 여기에 답변을 작성하세요 (print문 또는 주석으로)


### Q8. 혼동행렬 시각화하기 ⭐⭐⭐⭐

**문제**: 다음 혼동행렬을 Plotly 히트맵으로 시각화하세요.

```python
cm = np.array([[85, 15], [10, 90]])
labels = ['Negative', 'Positive']
```

In [ ]:
cm = np.array([[85, 15], [10, 90]])
labels = ['Negative', 'Positive']

# 여기에 코드를 작성하세요


### Q9. Learning Curve 그리기 ⭐⭐⭐⭐

**문제**: 유방암 데이터에 대해 DecisionTreeClassifier의 Learning Curve를 그리고, 과적합 여부를 판단하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q10. 종합 모델 평가 대시보드 ⭐⭐⭐⭐⭐

**문제**: 유방암 데이터에 대해 3개 모델(Logistic Regression, Random Forest, SVM)의 성능을 비교하는 대시보드를 만드세요.

**요구사항**:
1. 5-fold 교차검증으로 Accuracy, Precision, Recall, F1 계산
2. 결과를 DataFrame으로 정리
3. 막대 차트로 시각화

In [ ]:
# 여기에 코드를 작성하세요


---

## 📊 학습 정리

### Part 1: 분류 평가 지표 핵심 요약

| 지표 | 공식 | 언제 사용? |
|------|------|----------|
| **Accuracy** | (TP+TN) / 전체 | 클래스 균형 시 |
| **Precision** | TP / (TP+FP) | FP 비용이 클 때 (스팸) |
| **Recall** | TP / (TP+FN) | FN 비용이 클 때 (암진단) |
| **F1-Score** | 조화평균(P,R) | 불균형 데이터 |
| **Confusion Matrix** | 2x2 표 | 오분류 패턴 분석 |

### Part 2: 회귀 평가 지표 및 과적합 방지 핵심 요약

| 지표/기법 | 특징 | 활용 |
|----------|------|------|
| **MAE** | 절대 오차 평균 | 이상치에 덜 민감 |
| **RMSE** | 제곱 오차 → 루트 | 큰 오차 패널티 |
| **R-squared** | 설명력 (0~1) | 모델 비교 |
| **교차검증** | K-fold 평균 | 안정적 성능 추정 |
| **Learning Curve** | 학습량 vs 성능 | 과적합 진단 |

### 💡 실무 팁

1. **정확도 함정**: 불균형 데이터에서 정확도만 보면 안됨
2. **비용 기반 선택**: FP/FN 비용을 고려해 Precision/Recall 선택
3. **교차검증 필수**: 단일 분할 결과는 불안정
4. **Learning Curve**: 과적합 의심 시 반드시 확인
5. **여러 지표 종합**: 하나의 지표만 보지 말고 종합 판단